In [13]:
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# ---------- load & prep ----------
CSV_FILE = "gyro.csv"  # keep your file name here
raw = pd.read_csv(CSV_FILE)

# normalize column names we expect
time_col = "pc_timestamp_iso"
value_cols = ["x_dps", "y_dps", "z_dps"]

# coerce types
if time_col in raw.columns:
    raw[time_col] = pd.to_datetime(raw[time_col], errors="coerce")

for c in value_cols:
    if c in raw.columns:
        raw[c] = pd.to_numeric(raw[c], errors="coerce")

# keep only rows that have time and at least one numeric value
df = raw.dropna(subset=[time_col] + value_cols).reset_index(drop=True)

# ---------- figure factory ----------
def build_figure(kind: str, cols: list[str]):

    # guard rails
    cols = [c for c in cols if c in df.columns]
    if not cols:
        return px.line(title="Pick at least one variable")

    # common x-axis
    xaxis = time_col

    # time-series plots
    if kind in ("line", "scatter"):
        base = px.line() if kind == "line" else px.scatter()
        for v in cols:
            # create one trace per variable; attach directly for full control
            trace = (px.line(df, x=xaxis, y=v).data[0]
                     if kind == "line"
                     else px.scatter(df, x=xaxis, y=v).data[0])
            trace.name = v.replace("_dps", "").upper()
            base.add_trace(trace)
        base.update_layout(
            title=f"{kind.title()} — {', '.join([c.replace('_dps','').upper() for c in cols])}",
            xaxis_title="Time",
            yaxis_title="Degrees/sec"
        )
        return base

    # distribution-style plots use a melted frame
    melted = df[cols].melt(var_name="axis", value_name="value")
    if kind == "hist":
        return px.histogram(
            melted, x="value", color="axis", barmode="overlay", nbins=60,
            title=f"Histogram — {', '.join([c.replace('_dps','').upper() for c in cols])}"
        )
    if kind == "box":
        return px.box(
            melted, x="axis", y="value", points="outliers",
            title=f"Box Plot — {', '.join([c.replace('_dps','').upper() for c in cols])}"
        )

    return px.line(title="Unsupported chart type")

# ---------- app ----------
app = JupyterDash(__name__)
app.layout = html.Div(
    [
        html.H3("Gyroscope Explorer"),
        html.Div(
            [
                html.Div(
                    [
                        html.Label("Chart"),
                        dcc.Dropdown(
                            id="sel_chart",
                            options=[
                                {"label": "Line", "value": "line"},
                                {"label": "Scatter", "value": "scatter"},
                                {"label": "Histogram", "value": "hist"},
                                {"label": "Box", "value": "box"},
                            ],
                            value="line",
                            clearable=False,
                        ),
                    ],
                    style={"display": "inline-block", "width": "30%", "marginRight": "1%"},
                ),
                html.Div(
                    [
                        html.Label("Variables"),
                        dcc.Dropdown(
                            id="sel_vars",
                            options=[{"label": c.replace("_dps","").upper(), "value": c} for c in value_cols if c in df.columns],
                            value=[c for c in value_cols if c in df.columns],
                            multi=True,
                        ),
                    ],
                    style={"display": "inline-block", "width": "69%"},
                ),
            ],
            style={"marginBottom": "12px"},
        ),
        dcc.Graph(id="plot_area"),
    ],
    style={"maxWidth": "1100px", "margin": "0 auto"},
)

@app.callback(
    Output("plot_area", "figure"),
    Input("sel_chart", "value"),
    Input("sel_vars", "value"),
)
def _update_plot(kind, vars_in):
    # normalize input from dropdown
    chosen = vars_in if isinstance(vars_in, list) else [vars_in]
    return build_figure(kind, chosen)

app.run(mode="inline", port=8061, debug=True)



/opt/anaconda3/lib/python3.13/site-packages/dash/dash.py:634: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



1) Read and clean the CSV:
The code imports gyro.csv, converts the pc_timestamp_iso column to real datetimes for a nice time axis, and makes x_dps, y_dps, and z_dps numeric. Rows with missing required values are dropped so plots don't fail.

2) Build the app layout:
The layout features two dropdowns at the top—one to choose the chart type (Line, Scatter, Histogram, Box) and one to choose which axes (x, y, z) to plot. One big graph below is where the figure is displayed.

3) Handle interactions through a callback:
When I change either dropdown, Dash runs the callback update_figure(). That function calls build_figure(), which creates the proper Plotly figure based on the selected chart type and variables and returns it to the graph. This is how the app live updates without a page reload.

4) How plots are generated internally:
For Line and Scatter, the code loops through each selected variable (e.g., x_dps) and adds a separate trace to the figure with time on the x-axis.
For Histogram and Box, the data is melted into long format so that each axis is a category, which is how Plotly graphs individual colored distributions or boxes for x, y, and z.

What to look for in each plot (insight on my data)

Line / Scatter (time-series):
We can use these to visualize movement over time. Flat near-zero areas usually mean rest. Repeating spikes (especially on one axis) usually mean walking or some repeating motion. See which axis (X/Y/Z) has the greatest amplitude—that will tell you which direction saw the most rotation.

Histogram (distribution of values):
Shows how the rotation rates are distributed. A tight, centered cluster near zero shows lots of still time. A more dispersed distribution (and long "tails") shows more active motion with bigger swings. 

Box Plot (summary per axis):
Briefly examine median, quartiles, and outliers for both axes. If Y has higher median and wider box than X and Z, Y was more active overall (implying forward/back sway when walking). Outliers may be slight bumps or sharp turns.